In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import scipy.stats as sp
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import random

In [2]:
location_ids = [1, 168, 161, 201, 202, 6, 205, 171, 141, 179, 207, 163, 11, 180, 181,
       184, 15, 164, 213, 214, 165, 196, 522, 190, 189, 20]

In [3]:
year_id = 2019
age_group_ids = [2,3,4,5]

In [4]:
seqs = [1004, 1005, 1006, 1008, 1009, 1010, 1012, 1013, 1014, 1016, 1017, 1018, 1020, 1021, 1022, 1024, 1025, 1026, 1028, 1029, 1030, 1032, 1033, 1034, 1361, 1364, 1367, 1373, 1376, 1379, 1385, 1388, 1391, 1397, 1400, 1403, 1409, 1412, 1415, 1421, 1424, 1427, 1433, 1436, 1439, 1445, 1448, 1451, 5213, 5216, 5219, 5222, 5225, 5228, 5237, 5240, 5243, 5246, 5249, 5252, 5261, 5264, 5267, 5270, 5273, 5276,
  4985, 4988, 4991, 4994, 4997, 5000, 5009, 5012, 5015, 5678, 5681, 5684, 7214, 7217, 7220,
        4952, 4955, 4958, 4961, 4964, 4967, 4976, 4979, 4982, 5627, 5630, 5633, 7202, 7205, 7208,
        5393, 5396, 5399, 182, 183, 184, 240, 241, 242, 177, 178, 179, 144,145,146,172,173,174,525,526,527,1106,1107,1108,537,538,539,206,207,208,
        22989, 22990, 22991, 22992, 22993, 22999, 23000, 23001, 23002,
       23003, 23009, 23010, 23011, 23012, 23013,
       5567, 5570, 5573, 5579, 5582, 5585,
       23030, 23031, 23032, 23034, 23035, 23036, 23038, 23039, 23040,
       23042, 23043, 23044, 23046, 23047, 23048]
    

In [5]:
s = get_ids('sequela')
pd.DataFrame.to_csv(s[s['sequela_id'].isin(seqs)], 'out.csv')

# Under five, selected locations

In [6]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 year_id=2019,
                 measure_id=[3,5],
                 decomp_step='step5',
                 gbd_round_id=6)
ylds.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sequela_id,sex_id,year_id,metric_id
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,3,144,1,2019,3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,3,145,1,2019,3
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,3,146,1,2019,3
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,3,172,1,2019,3
4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,3,173,1,2019,3


In [7]:
# sum over causes
yld_sum = ylds.groupby(['location_id','sex_id','age_group_id','measure_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c]).reset_index()
yld_sum['parameter'] = 'Total iron responsive anemia'
yld_sum['metric_id'] = 'rate'
yld_sum['measure'] = np.where(yld_sum.measure_id==3, 'ylds', 'prevalence')
yld_sum.head()

,location_id,sex_id,age_group_id,measure_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,parameter,metric_id,measure
0,1,1,2,3,0.003992,0.003677,0.002815,0.003291,0.002532,0.003511,...,0.002811,0.002758,0.003320,0.001557,0.002465,0.002540,0.002312,Total iron responsive anemia,rate,ylds
1,1,1,2,5,0.200743,0.193420,0.181837,0.181523,0.176887,0.186832,...,0.182029,0.180130,0.183981,0.198579,0.175099,0.186115,0.170032,Total iron responsive anemia,rate,prevalence
2,1,1,3,3,0.023294,0.022104,0.017402,0.022141,0.018155,0.020508,...,0.018417,0.018546,0.020763,0.009605,0.016105,0.016702,0.015815,Total iron responsive anemia,rate,ylds
3,1,1,3,5,0.695343,0.681188,0.682700,0.688939,0.685925,0.673370,...,0.682245,0.693141,0.663559,0.698901,0.687547,0.688596,0.693989,Total iron responsive anemia,rate,prevalence
4,1,1,4,3,0.013691,0.014288,0.011304,0.013703,0.009904,0.013039,...,0.011801,0.011901,0.012714,0.006323,0.009873,0.010061,0.010332,Total iron responsive anemia,rate,ylds


In [8]:
pop_sum = get_population(location_id=location_ids,
                     sex_id=[1,2],
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=age_group_ids).drop(columns=['year_id','run_id'])
pop_sum.head()

,age_group_id,location_id,sex_id,population
0,2,1,1,1.330245e+06
1,3,1,1,3.951181e+06
2,4,1,1,6.292360e+07
3,5,1,1,2.741937e+08
4,2,1,2,1.241845e+06


In [9]:
counts = yld_sum.merge(pop_sum, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    counts[f'draw_{i}'] = counts[f'draw_{i}'] * counts['population']
    
counts['metric_id'] = 'count'
# sum over age groups and sexes for total under 5 population for each location and measure
counts = counts.groupby([c for c in counts.columns if 'draw' not in c and c not in ['sex_id','population','age_group_id']]).sum().drop(columns='age_group_id').reset_index()
counts['sex_id'] = 3
counts['age_group_id'] = 1
counts.head()

,location_id,measure_id,parameter,metric_id,measure,sex_id,draw_0,draw_1,draw_10,draw_100,...,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,population,age_group_id
0,1,3,Total iron responsive anemia,count,ylds,3,7.989029e+06,7.585250e+06,5.992380e+06,7.537664e+06,...,6.945814e+06,6.379412e+06,6.402699e+06,7.109910e+06,3.352733e+06,5.513215e+06,5.493529e+06,5.425313e+06,6.628427e+08,1
1,1,5,Total iron responsive anemia,count,prevalence,3,2.291708e+08,2.275500e+08,2.241093e+08,2.284715e+08,...,2.232003e+08,2.290391e+08,2.274000e+08,2.213082e+08,2.298323e+08,2.273891e+08,2.255495e+08,2.306073e+08,6.628427e+08,1
2,6,3,Total iron responsive anemia,count,ylds,3,1.299952e+05,1.218326e+05,8.196672e+04,1.531845e+05,...,1.197661e+05,1.191495e+05,1.050212e+05,1.256943e+05,4.955670e+04,1.200867e+05,7.196413e+04,1.217915e+05,8.149086e+07,1
3,6,5,Total iron responsive anemia,count,prevalence,3,5.748553e+06,5.493788e+06,4.823885e+06,6.568525e+06,...,5.795847e+06,6.313868e+06,5.724151e+06,5.886661e+06,5.292320e+06,6.792652e+06,5.018238e+06,7.482364e+06,8.149086e+07,1
4,11,3,Total iron responsive anemia,count,ylds,3,6.599940e+04,8.853337e+04,6.152083e+04,7.312906e+04,...,6.711963e+04,8.804908e+04,6.877152e+04,5.796316e+04,2.390642e+04,4.863280e+04,5.285984e+04,4.098323e+04,1.970635e+07,1


In [10]:
rates = pd.DataFrame.copy(counts)
for i in list(range(0,1000)):
    rates[f'draw_{i}'] = rates[f'draw_{i}'] / rates['population']
rates['metric_id'] = 'rate'
rates.head()

,location_id,measure_id,parameter,metric_id,measure,sex_id,draw_0,draw_1,draw_10,draw_100,...,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,population,age_group_id
0,1,3,Total iron responsive anemia,rate,ylds,3,0.012053,0.011444,0.009040,0.011372,...,0.010479,0.009624,0.009659,0.010726,0.005058,0.008318,0.008288,0.008185,6.628427e+08,1
1,1,5,Total iron responsive anemia,rate,prevalence,3,0.345739,0.343294,0.338103,0.344684,...,0.336732,0.345541,0.343068,0.333877,0.346737,0.343051,0.340276,0.347906,6.628427e+08,1
2,6,3,Total iron responsive anemia,rate,ylds,3,0.001595,0.001495,0.001006,0.001880,...,0.001470,0.001462,0.001289,0.001542,0.000608,0.001474,0.000883,0.001495,8.149086e+07,1
3,6,5,Total iron responsive anemia,rate,prevalence,3,0.070542,0.067416,0.059195,0.080604,...,0.071123,0.077479,0.070243,0.072237,0.064944,0.083355,0.061580,0.091818,8.149086e+07,1
4,11,3,Total iron responsive anemia,rate,ylds,3,0.003349,0.004493,0.003122,0.003711,...,0.003406,0.004468,0.003490,0.002941,0.001213,0.002468,0.002682,0.002080,1.970635e+07,1


In [11]:
l = get_ids('location')
l = l.loc[l.location_id.isin(location_ids)]
l = l[['location_id','location_name']]

In [12]:
ira = pd.concat([rates, counts], ignore_index=True)
ira = ira.merge(l, on='location_id').drop(columns='population')
ira = ira.set_index([c for c in ira.columns if 'draw' not in c])
ira = ira.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1).rename(columns={'2.5%':'lower','97.5%':'upper'})
ira = ira[['mean','lower','upper']]
ira = ira.reset_index()
ira.head()

,location_id,measure_id,parameter,metric_id,measure,sex_id,age_group_id,location_name,mean,lower,upper
0,1,3,Total iron responsive anemia,rate,ylds,3,1,Global,1.030491e-02,6.890595e-03,1.491708e-02
1,1,5,Total iron responsive anemia,rate,prevalence,3,1,Global,3.412452e-01,3.332895e-01,3.490746e-01
2,1,3,Total iron responsive anemia,count,ylds,3,1,Global,6.830531e+06,4.567381e+06,9.887679e+06
3,1,5,Total iron responsive anemia,count,prevalence,3,1,Global,2.261919e+08,2.209185e+08,2.313816e+08
4,6,3,Total iron responsive anemia,rate,ylds,3,1,China,1.546049e-03,8.687570e-04,2.448056e-03



# Women of reproductive age in selected locations

In [13]:
ages = [7, 8, 9, 10, 11, 12, 13, 14, 15] # 10-54

In [14]:
ylds_wra = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=ages,
                 year_id=2019,
                 measure_id=[3,5],
                     sex_id=2,
                 decomp_step='step5',
                 gbd_round_id=6)
ylds_wra.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sequela_id,sex_id,year_id,metric_id
0,7,6.285875e-07,8.876883e-07,9.279934e-07,0.000001,4.709083e-07,6.699409e-07,6.914293e-07,8.533791e-07,0.000003,...,2.631115e-07,7.421502e-07,9.539510e-07,4.735020e-07,1,3,144,2,2019,3
1,7,4.427225e-05,4.478210e-05,3.173890e-05,0.000039,3.391062e-05,3.574721e-05,4.206319e-05,2.975047e-05,0.000082,...,2.400067e-05,3.669814e-05,4.859811e-05,3.856409e-05,1,3,145,2,2019,3
2,7,1.177433e-05,6.095171e-06,4.957820e-06,0.000006,9.284979e-06,7.864537e-06,7.005875e-06,7.034097e-06,0.000014,...,9.139631e-06,9.520524e-06,8.167625e-06,9.370701e-06,1,3,146,2,2019,3
3,7,9.613952e-07,1.423029e-06,1.502879e-06,0.000002,8.141798e-07,1.390992e-06,1.016309e-06,1.790696e-06,0.000003,...,4.290841e-07,9.971721e-07,1.028192e-06,6.694276e-07,1,3,172,2,2019,3
4,7,7.649160e-05,8.353950e-05,5.894396e-05,0.000071,6.369756e-05,7.581940e-05,7.990197e-05,5.999081e-05,0.000083,...,3.501279e-05,5.610574e-05,5.675986e-05,4.886782e-05,1,3,173,2,2019,3


In [15]:
# sum over causes
yld_sum_wra = ylds_wra.groupby(['location_id','sex_id','age_group_id','measure_id']).sum()
yld_sum_wra = yld_sum_wra.drop(columns=[c for c in yld_sum_wra.columns if 'draw' not in c]).reset_index()
yld_sum_wra['parameter'] = 'Total iron responsive anemia'
yld_sum_wra['metric_id'] = 'rate'
yld_sum_wra['measure'] = np.where(yld_sum_wra.measure_id==3, 'ylds', 'prevalence')
yld_sum_wra.head()

,location_id,sex_id,age_group_id,measure_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,parameter,metric_id,measure
0,1,2,7,3,0.008585,0.007903,0.006434,0.007520,0.006713,0.007743,...,0.007079,0.006797,0.008402,0.003705,0.005678,0.005986,0.005899,Total iron responsive anemia,rate,ylds
1,1,2,7,5,0.182192,0.181377,0.180771,0.169972,0.184604,0.184583,...,0.187327,0.178589,0.189536,0.184504,0.171154,0.182053,0.184581,Total iron responsive anemia,rate,prevalence
2,1,2,8,3,0.008251,0.006903,0.006196,0.007498,0.006216,0.007072,...,0.006516,0.006692,0.007272,0.003401,0.005331,0.005476,0.005628,Total iron responsive anemia,rate,ylds
3,1,2,8,5,0.232165,0.218208,0.224901,0.217363,0.226183,0.222091,...,0.222313,0.224496,0.219031,0.217690,0.216504,0.217543,0.220903,Total iron responsive anemia,rate,prevalence
4,1,2,9,3,0.006979,0.006608,0.005435,0.006785,0.005331,0.006047,...,0.005518,0.006078,0.006147,0.003117,0.004829,0.005090,0.005003,Total iron responsive anemia,rate,ylds


In [16]:
pop = get_population(location_id=location_ids,
                     sex_id=2,
                    gbd_round_id=6,
                    decomp_step='step4',
                     year_id=2019,
                    age_group_id=ages).drop(columns=['year_id','run_id'])
pop.head()

,age_group_id,location_id,sex_id,population
0,7,1,2,3.108525e+08
1,8,1,2,3.017589e+08
2,9,1,2,2.957762e+08
3,10,1,2,3.006931e+08
4,11,1,2,2.985578e+08


In [17]:
counts = yld_sum_wra.merge(pop, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    counts[f'draw_{i}'] = counts[f'draw_{i}'] * counts['population']   
counts['metric_id'] = 'count'
# sum over age groups for total WRA population for each location and measure
counts = counts.groupby([c for c in counts.columns if 'draw' not in c and c not in ['population','age_group_id']]).sum().drop(columns='age_group_id').reset_index()
counts['age_group_id'] = 169
counts.head()

,location_id,sex_id,measure_id,parameter,metric_id,measure,draw_0,draw_1,draw_10,draw_100,...,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,population,age_group_id
0,1,2,3,Total iron responsive anemia,count,ylds,1.859513e+07,1.739472e+07,1.429148e+07,1.758637e+07,...,1.702968e+07,1.510684e+07,1.593890e+07,1.691335e+07,8.462072e+06,1.330488e+07,1.338131e+07,1.350215e+07,2.475433e+09,169
1,1,2,5,Total iron responsive anemia,count,prevalence,4.877657e+08,4.849668e+08,4.802927e+08,4.828118e+08,...,4.916533e+08,4.863026e+08,4.931673e+08,4.697352e+08,4.900129e+08,4.853823e+08,4.946305e+08,4.969920e+08,2.475433e+09,169
2,6,2,3,Total iron responsive anemia,count,ylds,9.547880e+05,9.656289e+05,6.767986e+05,9.297371e+05,...,1.230936e+06,8.917979e+05,9.589747e+05,8.231748e+05,4.786299e+05,8.675880e+05,7.312954e+05,9.036982e+05,4.456938e+08,169
3,6,2,5,Total iron responsive anemia,count,prevalence,3.156062e+07,3.295448e+07,2.852559e+07,3.121515e+07,...,4.015882e+07,3.458614e+07,3.508780e+07,2.806764e+07,3.388784e+07,3.671515e+07,3.232281e+07,3.867451e+07,4.456938e+08,169
4,11,2,3,Total iron responsive anemia,count,ylds,7.223983e+05,6.087747e+05,4.823382e+05,6.365435e+05,...,5.184120e+05,5.080286e+05,5.544738e+05,5.829163e+05,3.173091e+05,4.180134e+05,3.902208e+05,5.119004e+05,8.921335e+07,169


In [18]:
rates = pd.DataFrame.copy(counts)
for i in list(range(0,1000)):
    rates[f'draw_{i}'] = rates[f'draw_{i}'] / rates['population']
rates['metric_id'] = 'rate'
rates.head()

,location_id,sex_id,measure_id,parameter,metric_id,measure,draw_0,draw_1,draw_10,draw_100,...,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,population,age_group_id
0,1,2,3,Total iron responsive anemia,rate,ylds,0.007512,0.007027,0.005773,0.007104,...,0.006879,0.006103,0.006439,0.006832,0.003418,0.005375,0.005406,0.005454,2.475433e+09,169
1,1,2,5,Total iron responsive anemia,rate,prevalence,0.197043,0.195912,0.194024,0.195041,...,0.198613,0.196452,0.199225,0.189759,0.197950,0.196080,0.199816,0.200770,2.475433e+09,169
2,6,2,3,Total iron responsive anemia,rate,ylds,0.002142,0.002167,0.001519,0.002086,...,0.002762,0.002001,0.002152,0.001847,0.001074,0.001947,0.001641,0.002028,4.456938e+08,169
3,6,2,5,Total iron responsive anemia,rate,prevalence,0.070812,0.073940,0.064003,0.070037,...,0.090104,0.077601,0.078726,0.062975,0.076034,0.082378,0.072522,0.086774,4.456938e+08,169
4,11,2,3,Total iron responsive anemia,rate,ylds,0.008097,0.006824,0.005407,0.007135,...,0.005811,0.005695,0.006215,0.006534,0.003557,0.004686,0.004374,0.005738,8.921335e+07,169


In [19]:
ira_wra = pd.concat([rates, counts], ignore_index=True)
ira_wra = ira_wra.merge(l, on='location_id').drop(columns='population')
ira_wra = ira_wra.set_index([c for c in ira_wra.columns if 'draw' not in c])
ira_wra = ira_wra.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1).rename(columns={'2.5%':'lower','97.5%':'upper'})
ira_wra = ira_wra[['mean','lower','upper']]
ira_wra = ira_wra.reset_index()
ira_wra.head()

,location_id,sex_id,measure_id,parameter,metric_id,measure,age_group_id,location_name,mean,lower,upper
0,1,2,3,Total iron responsive anemia,rate,ylds,169,Global,6.488779e-03,4.487159e-03,9.090281e-03
1,1,2,5,Total iron responsive anemia,rate,prevalence,169,Global,1.954898e-01,1.905729e-01,2.001988e-01
2,1,2,3,Total iron responsive anemia,count,ylds,169,Global,1.606254e+07,1.110766e+07,2.250238e+07
3,1,2,5,Total iron responsive anemia,count,prevalence,169,Global,4.839220e+08,4.717503e+08,4.955787e+08
4,6,2,3,Total iron responsive anemia,rate,ylds,169,China,1.997833e-03,1.313038e-03,2.839520e-03


In [20]:
final_cols = ['parameter', 'measure_id','measure', 'metric_id', 'location_id', 'age_group_id',
       'sex_id', 'location_name', 'sex', 'age', 'mean', 'lower', 'upper']

In [21]:
final_ira = pd.concat([ira_wra, ira], ignore_index=True)
final_ira['sex'] = np.where(final_ira.sex_id == 3, 'Both', 'Female')
final_ira['age'] = np.where(final_ira.age_group_id == 1, 'Under Five', 'Reproductive age')
final_ira['measire'] = np.where(final_ira.measure_id==3, 'ylds', 'prevalence')
final_ira = final_ira[final_cols]
final_ira.to_csv('ira_anemia_prevalence_and_counts.csv')
final_ira.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v103/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,parameter,measure_id,measure,metric_id,location_id,age_group_id,sex_id,location_name,sex,age,mean,lower,upper
0,Total iron responsive anemia,3,ylds,rate,1,169,2,Global,Female,Reproductive age,6.488779e-03,4.487159e-03,9.090281e-03
1,Total iron responsive anemia,5,prevalence,rate,1,169,2,Global,Female,Reproductive age,1.954898e-01,1.905729e-01,2.001988e-01
2,Total iron responsive anemia,3,ylds,count,1,169,2,Global,Female,Reproductive age,1.606254e+07,1.110766e+07,2.250238e+07
3,Total iron responsive anemia,5,prevalence,count,1,169,2,Global,Female,Reproductive age,4.839220e+08,4.717503e+08,4.955787e+08
4,Total iron responsive anemia,3,ylds,rate,6,169,2,China,Female,Reproductive age,1.997833e-03,1.313038e-03,2.839520e-03


# Burdenator estimates

In [22]:
mat_burden_raw = get_draws(
        gbd_id_type=['rei_id', 'cause_id'], # Types must match gbd_id's
        gbd_id=[95, 366], # iron def, maternal disorders
        source='burdenator',
        measure_id=2, # dalys
        metric_id=1, # number
        location_id=location_ids,
        year_id=2019,
    age_group_id=ages,
        sex_id=[2], # Sex aggregates not available
        gbd_round_id=6,
        status='best',
        decomp_step='step5',
    )
mat_burden_raw.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,rei_id,sex_id,year_id
0,7,366,12119.604302,19443.432191,21611.427124,18809.844215,10733.733228,19474.288146,12500.252382,11760.220344,...,15615.958072,10881.233184,13415.293314,11694.688854,1,2,1,95,2,2019
1,8,366,271423.538755,355678.383836,415699.873912,455242.618361,270196.657577,383776.024252,274926.282206,245097.904329,...,370377.975344,284552.059279,353873.962396,334168.193292,1,2,1,95,2,2019
2,9,366,528179.721962,746531.089708,743630.874715,847137.384992,456141.967843,718647.096414,496015.037760,459468.986710,...,712618.452515,524767.305914,677203.297696,630987.767523,1,2,1,95,2,2019
3,10,366,540937.468030,707847.445982,741599.644674,868017.788149,485108.268665,684967.264005,512155.899508,431129.990883,...,691619.470708,504820.282674,652379.909436,562515.779181,1,2,1,95,2,2019
4,11,366,493040.436822,631029.623580,607754.732527,733541.787587,413441.757547,604503.303875,433105.613628,381073.319945,...,600942.434636,453490.369242,563925.377097,489195.422826,1,2,1,95,2,2019


In [33]:
mat_burden = mat_burden_raw.groupby(['location_id','measure_id','metric_id','rei_id','year_id','cause_id','sex_id'], as_index=False).sum().drop(columns=['age_group_id'])
#mat_burden = mat_burden.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1).rename(columns={'2.5%':'lower','97.5%':'upper'})
#mat_burden = mat_burden[['mean','lower','upper']].reset_index()
mat_burden['metric_id'] = 'count'
mat_burden.head()

,location_id,measure_id,metric_id,rei_id,year_id,cause_id,sex_id,draw_0,draw_1,draw_10,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,1,2,count,95,2019,366,2,2.586464e+06,3.412007e+06,3.510536e+06,...,1.953944e+06,3.362882e+06,3.509615e+06,2.737190e+06,2.240182e+06,2.131786e+06,3.293110e+06,2.447958e+06,3.063591e+06,2.759295e+06
1,6,2,count,95,2019,366,2,1.592559e+04,2.345098e+04,2.039804e+04,...,1.396210e+04,2.383112e+04,2.302099e+04,1.973767e+04,1.708851e+04,1.357845e+04,1.833070e+04,1.673002e+04,1.885510e+04,2.033403e+04
2,11,2,count,95,2019,366,2,4.562846e+04,5.708146e+04,6.967349e+04,...,2.801593e+04,6.543340e+04,6.151275e+04,4.810679e+04,5.727390e+04,3.965646e+04,7.165324e+04,4.580708e+04,6.876884e+04,6.591841e+04
3,15,2,count,95,2019,366,2,2.183573e+04,2.616685e+04,2.617762e+04,...,1.304211e+04,2.050326e+04,2.833661e+04,2.283352e+04,1.777882e+04,1.834445e+04,2.264813e+04,1.971836e+04,1.972938e+04,1.470944e+04
4,20,2,count,95,2019,366,2,2.404539e+03,3.950666e+03,3.812177e+03,...,1.685713e+03,3.426143e+03,2.869611e+03,2.874473e+03,2.326428e+03,2.417602e+03,2.640399e+03,2.505255e+03,3.434226e+03,2.660900e+03


In [34]:
mat_rates = mat_burden.merge(pop.groupby(['location_id','sex_id'], as_index=False).sum().drop(columns='age_group_id'), 
                             on=['location_id','sex_id'])
for i in list(range(0,1000)):
    mat_rates[f'draw_{i}'] = mat_rates[f'draw_{i}'] / mat_rates['population']
mat_rates = mat_rates.drop(columns='population')
mat_rates['metric_id'] = 'rate'
mat_rates.head()

,location_id,measure_id,metric_id,rei_id,year_id,cause_id,sex_id,draw_0,draw_1,draw_10,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,1,2,rate,95,2019,366,2,0.001045,0.001378,0.001418,...,0.000789,0.001359,0.001418,0.001106,0.000905,0.000861,0.001330,0.000989,0.001238,0.001115
1,6,2,rate,95,2019,366,2,0.000036,0.000053,0.000046,...,0.000031,0.000053,0.000052,0.000044,0.000038,0.000030,0.000041,0.000038,0.000042,0.000046
2,11,2,rate,95,2019,366,2,0.000511,0.000640,0.000781,...,0.000314,0.000733,0.000690,0.000539,0.000642,0.000445,0.000803,0.000513,0.000771,0.000739
3,15,2,rate,95,2019,366,2,0.001149,0.001377,0.001377,...,0.000686,0.001079,0.001491,0.001201,0.000936,0.000965,0.001192,0.001038,0.001038,0.000774
4,20,2,rate,95,2019,366,2,0.000074,0.000121,0.000117,...,0.000052,0.000105,0.000088,0.000088,0.000071,0.000074,0.000081,0.000077,0.000105,0.000082


In [37]:
mat_dalys = pd.concat([mat_burden, mat_rates], ignore_index=True).drop(columns='rei_id')
mat_dalys = mat_dalys.merge(l, on='location_id')
mat_dalys['measure'] = 'dalys'
mat_dalys['parameter'] = 'Maternal disorder burden attributable to iron deficiency'
mat_dalys['sex'] = 'female'
mat_dalys['age'] = 'Reproductive age'
mat_dalys['age_group_id'] =169
mat_dalys = mat_dalys.set_index([c for c in mat_dalys.columns if 'draw' not in c])
mat_dalys = mat_dalys.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1).reset_index().rename(columns={'2.5%':'lower','97.5%':'upper'})
mat_dalys = mat_dalys[final_cols]
#mat_dalys.to_csv('maternal_disorder_burden.csv')
mat_dalys.head()

,parameter,measure_id,measure,metric_id,location_id,age_group_id,sex_id,location_name,sex,age,mean,lower,upper
0,Maternal disorder burden attributable to iron ...,2,dalys,count,1,169,2,Global,female,Reproductive age,2.728445e+06,977285.435299,4.524662e+06
1,Maternal disorder burden attributable to iron ...,2,dalys,rate,1,169,2,Global,female,Reproductive age,1.102209e-03,0.000395,1.827827e-03
2,Maternal disorder burden attributable to iron ...,2,dalys,count,6,169,2,China,female,Reproductive age,1.755876e+04,5961.172412,3.115521e+04
3,Maternal disorder burden attributable to iron ...,2,dalys,rate,6,169,2,China,female,Reproductive age,3.939647e-05,0.000013,6.990271e-05
4,Maternal disorder burden attributable to iron ...,2,dalys,count,11,169,2,Indonesia,female,Reproductive age,4.959363e+04,17455.589311,8.553133e+04


In [26]:
mat_dalys.metric_id.unique()

array(['count', 'rate'], dtype=object)

In [27]:
ira_wra = pd.concat([rates, counts], ignore_index=True)
ira_wra = ira_wra.merge(l, on='location_id').drop(columns='population')
ira_wra.head()

,location_id,sex_id,measure_id,parameter,metric_id,measure,draw_0,draw_1,draw_10,draw_100,...,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_group_id,location_name
0,1,2,3,Total iron responsive anemia,rate,ylds,7.511872e-03,7.026941e-03,5.773325e-03,7.104360e-03,...,6.879478e-03,6.102708e-03,6.438835e-03,6.832483e-03,3.418422e-03,5.374770e-03,5.405647e-03,5.454462e-03,169,Global
1,1,2,5,Total iron responsive anemia,rate,prevalence,1.970426e-01,1.959119e-01,1.940237e-01,1.950414e-01,...,1.986131e-01,1.964515e-01,1.992247e-01,1.897588e-01,1.979504e-01,1.960798e-01,1.998158e-01,2.007697e-01,169,Global
2,1,2,3,Total iron responsive anemia,count,ylds,1.859513e+07,1.739472e+07,1.429148e+07,1.758637e+07,...,1.702968e+07,1.510684e+07,1.593890e+07,1.691335e+07,8.462072e+06,1.330488e+07,1.338131e+07,1.350215e+07,169,Global
3,1,2,5,Total iron responsive anemia,count,prevalence,4.877657e+08,4.849668e+08,4.802927e+08,4.828118e+08,...,4.916533e+08,4.863026e+08,4.931673e+08,4.697352e+08,4.900129e+08,4.853823e+08,4.946305e+08,4.969920e+08,169,Global
4,6,2,3,Total iron responsive anemia,rate,ylds,2.142251e-03,2.166575e-03,1.518528e-03,2.086044e-03,...,2.761842e-03,2.000920e-03,2.151645e-03,1.846951e-03,1.073899e-03,1.946601e-03,1.640802e-03,2.027621e-03,169,China


In [38]:
mat_dalys = pd.concat([mat_burden, mat_rates], ignore_index=True).drop(columns='rei_id')
mat_dalys = mat_dalys.merge(l, on='location_id')
mat_dalys['measure'] = 'dalys'
mat_dalys['parameter'] = 'Maternal disorder burden attributable to iron deficiency'
mat_dalys['sex'] = 'female'
mat_dalys['age'] = 'Reproductive age'
mat_dalys['age_group_id'] =169
mat_dalys.head()

,location_id,measure_id,metric_id,year_id,cause_id,sex_id,draw_0,draw_1,draw_10,draw_100,...,draw_996,draw_997,draw_998,draw_999,location_name,measure,parameter,sex,age,age_group_id
0,1,2,count,2019,366,2,2.586464e+06,3.412007e+06,3.510536e+06,4.085623e+06,...,3.293110e+06,2.447958e+06,3.063591e+06,2.759295e+06,Global,dalys,Maternal disorder burden attributable to iron ...,female,Reproductive age,169
1,1,2,rate,2019,366,2,1.044853e-03,1.378348e-03,1.418150e-03,1.650468e-03,...,1.330317e-03,9.889010e-04,1.237598e-03,1.114672e-03,Global,dalys,Maternal disorder burden attributable to iron ...,female,Reproductive age,169
2,6,2,count,2019,366,2,1.592559e+04,2.345098e+04,2.039804e+04,2.713924e+04,...,1.833070e+04,1.673002e+04,1.885510e+04,2.033403e+04,China,dalys,Maternal disorder burden attributable to iron ...,female,Reproductive age,169
3,6,2,rate,2019,366,2,3.573213e-05,5.261679e-05,4.576694e-05,6.089211e-05,...,4.112847e-05,3.753702e-05,4.230506e-05,4.562332e-05,China,dalys,Maternal disorder burden attributable to iron ...,female,Reproductive age,169
4,11,2,count,2019,366,2,4.562846e+04,5.708146e+04,6.967349e+04,7.099165e+04,...,7.165324e+04,4.580708e+04,6.876884e+04,6.591841e+04,Indonesia,dalys,Maternal disorder burden attributable to iron ...,female,Reproductive age,169


In [44]:
[c for c in mat_dalys.columns if 'draw' not in c]

['location_id',
 'measure_id',
 'metric_id',
 'year_id',
 'cause_id',
 'sex_id',
 'location_name',
 'measure',
 'parameter',
 'sex',
 'age',
 'age_group_id']

In [45]:
ira_wra['age'] = 169
ira_wra['sex'] = 2
[c for c in ira_wra.columns if 'draw' not in c]

['location_id',
 'sex_id',
 'measure_id',
 'parameter',
 'metric_id',
 'measure',
 'age_group_id',
 'location_name',
 'age',
 'sex']

In [79]:
ira_wra['age'] = 'Reproductive age'
ira_wra['sex'] = 'female'
tot_iron = pd.concat([mat_dalys, ira_wra], ignore_index=False).drop(columns=['year_id','cause_id'])
tot_iron['parameter'] = 'Total iron deficiency burden (including anemia and maternal disorders)'
tot_iron = tot_iron.loc[tot_iron.measure_id!=5]
tot_iron = tot_iron.groupby(['parameter','metric_id','location_id','age_group_id','sex_id','sex','location_name','age']).sum().reset_index()
tot_iron['measure_id'] = 2
tot_iron['measure'] = 'dalys'
tot_iron = tot_iron.set_index([c for c in tot_iron.columns if 'draw' not in c])
tot_iron = tot_iron.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1).reset_index().rename(columns={'2.5%':'lower','97.5%':'upper'})
tot_iron = tot_iron[final_cols]
tot_iron.to_csv('total_iron_burden.csv')
tot_iron.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v103/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,parameter,measure_id,measure,metric_id,location_id,age_group_id,sex_id,location_name,sex,age,mean,lower,upper
0,Total iron deficiency burden (including anemia...,2,dalys,count,1,169,2,Global,female,Reproductive age,1.879098e+07,1.305012e+07,2.573365e+07
1,Total iron deficiency burden (including anemia...,2,dalys,count,6,169,2,China,female,Reproductive age,9.079805e+05,6.023989e+05,1.288013e+06
2,Total iron deficiency burden (including anemia...,2,dalys,count,11,169,2,Indonesia,female,Reproductive age,6.009041e+05,4.121632e+05,8.466992e+05
3,Total iron deficiency burden (including anemia...,2,dalys,count,15,169,2,Myanmar,female,Reproductive age,2.025691e+05,1.369511e+05,2.808954e+05
4,Total iron deficiency burden (including anemia...,2,dalys,count,20,169,2,Viet Nam,female,Reproductive age,9.993928e+04,6.594091e+04,1.471027e+05


In [74]:
tot_iron.sex.unique()

array(['female'], dtype=object)

In [75]:
len(tot_iron)

52

In [78]:
len(tot_iron.location_id.unique()) * 2

52